In [1]:
# import required libriaries
from pathlib import Path
from helpers import *
import json

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v7.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2020-21']

# name for current season
season_names = ['2021']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2021', 11, range(13,33))

In [4]:
all_players[all_players['full_name'] == 'Ben Davies']

,full_name,play_proba_2021,position_2021,id_2021,cost_2021,team_2021,web_name_2021
368,Ben Davies,100,2,653,48,14,Davies
369,Ben Davies,0,2,395,46,6,Davies


In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles
108188,Thomas McGill,28,1,0,Brighton and Hove Albion,Southampton,0.579470,0.583899,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108189,Thomas Partey,28,3,90,Arsenal,Tottenham Hotspur,1.279916,1.586900,True,2,...,0.0,0.1,3.0,0.0,49.0,63.0,77.8,44.0,0.0,1.0
108190,Timothy Castagne,28,2,90,Leicester City,Sheffield United,1.109292,0.356167,True,6,...,0.0,0.0,2.0,0.0,56.0,62.0,90.3,50.0,1.0,1.0
108191,Timothy Fosu-Mensah,28,2,0,Manchester United,West Ham United,1.680487,0.652079,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108192,Timo Werner,28,4,21,Chelsea,Leeds,1.817663,0.500335,False,1,...,0.1,0.0,0.0,0.0,3.0,5.0,60.0,5.0,0.0,1.0
108193,Tim Ream,28,2,0,Fulham,Manchester City,0.520031,2.400862,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108194,Toby Alderweireld,28,2,90,Tottenham Hotspur,Arsenal,1.586900,1.279916,False,1,...,0.0,0.0,0.0,0.0,64.0,71.0,90.1,59.0,0.0,0.0
108195,Tomas Soucek,28,3,90,West Ham United,Manchester United,0.652079,1.680487,False,2,...,0.5,0.0,2.0,0.0,25.0,32.0,78.1,23.0,1.0,3.0
108196,Tommy Doyle,28,3,0,Manchester City,Fulham,2.400862,0.520031,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108197,Tom Cairney,28,3,0,Fulham,Manchester City,0.520031,2.400862,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

29

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron Connolly,29,4,0,Brighton and Hove Albion,Newcastle United,0.579673,0.589233,True,0,...,23344,0,3,0.0,-78,564,642,0,2021-03-20T20:00:00Z,2021
1,Aaron Cresswell,29,2,90,West Ham United,Arsenal,0.652307,1.280364,True,0,...,2475507,3,3,0.0,204684,221244,16560,1,2021-03-21T15:00:00Z,2021
2,Aaron Mooy,29,3,0,Brighton and Hove Albion,Newcastle United,0.579673,0.589233,True,0,...,832,0,3,0.0,-2,0,2,0,2021-03-20T20:00:00Z,2021
3,Aboubakar Kamara,29,3,0,Fulham,Leeds,0.520213,0.500510,True,0,...,3138,2,1,0.0,-12,0,12,0,2021-03-19T20:00:00Z,2021
4,Adam Forshaw,29,3,0,Leeds,Fulham,0.500510,0.520213,False,0,...,627,2,1,0.0,-47,28,75,0,2021-03-19T20:00:00Z,2021
5,Adam Lallana,29,3,90,Brighton and Hove Albion,Newcastle United,0.579673,0.589233,True,3,...,59767,0,3,2.0,17366,19246,1880,0,2021-03-20T20:00:00Z,2021
6,Adam Webster,29,2,0,Brighton and Hove Albion,Newcastle United,0.579673,0.589233,True,0,...,71855,0,3,0.0,-3408,58,3466,0,2021-03-20T20:00:00Z,2021
7,Ademipo Odubeko,29,4,0,West Ham United,Arsenal,0.652307,1.280364,True,0,...,3372,3,3,0.0,-30,329,359,0,2021-03-21T15:00:00Z,2021
8,Ademola Lookman,29,3,90,Fulham,Leeds,0.520213,0.500510,True,5,...,359227,2,1,61.0,104271,113097,8826,0,2021-03-19T20:00:00Z,2021
9,Ahmed El Mohamady,29,2,0,Aston Villa,Tottenham Hotspur,0.885365,1.587455,True,0,...,36572,2,0,0.0,-930,1199,2129,0,2021-03-21T19:30:00Z,2021


In [10]:
df_latest.shape

(277, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [12]:
last_play_proba.head()

,player,play_proba
0,Emiliano Martínez,1.0
1,Tom Heaton,1.0
2,Ahmed El Mohamady,1.0
3,Henri Lansbury,0.0
4,Neil Taylor,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [18]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2020-2021
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
Rúben Dias
Rodri
Raheem Sterling
João Cancelo
Kevin De Bruyne
İlkay Gündoğan
Bernardo Silva
Riyad Mahrez
John Stones
Kyle Walker
Gabriel Jesus
Phil Foden
Oleksandr Zinchenko
Aymeric Laporte
Ferrán Torres
Fernandinho
Benjamin Mendy
Nathan Aké
Sergio Agüero
Eric García
Zack Steffen
Liam Delap
Manchester Utd
https://fbref.com/en/squads/19538871/Manchester-United-Stats
Harry Maguire
Bruno Fernandes
Aaron Wan-Bissaka
Marcus Rashford
David de Gea
Luke Shaw
Victor Lindelöf
Fred
Scott McTominay
Anthony Martial
Paul Pogba
Mason Greenwood
Nemanja Matić
Edinson Cavani
Daniel James
Eric Bailly
Alex Telles
Dean Henderson
Juan Mata
Donny van de Beek
Axel Tuanzebe
Timothy Fosu-Mensah
Odion Ighalo
Brandon Williams
Shola Shoretire
Leicester City
https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats
Kasper Schmeichel
Youri Tielemans
James Justin
Jonny Evans
J

Charlie Austin
Sam Field
Rekeem Harper
Sheffield Utd
https://fbref.com/en/squads/1df6b87e/Sheffield-United-Stats
Aaron Ramsdale
Chris Basham
George Baldock
John Egan
Enda Stevens
Ethan Ampadu
John Lundstram
John Fleck
David McGoldrick
Oliver Norwood
Oliver Burke
Sander Berge
Oliver McBurnie
Ben Osborn
Rhian Brewster
Kean Bryan
Jayden Bogle
Billy Sharp
Max Lowe
Jack Robinson
Phil Jagielka
Lys Mousset
Jack O'Connell
Iliman Ndiaye
Antwoine Hackford


In [19]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [20]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [21]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [22]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [26]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2020-2021,Ederson,2020-09-21,Manchester City,Premier League,2,0,0,36,0,...,0.0,0,0,30,36,83.3,27,0,0,Ederson Santana de Moraes
1,2020-2021,Ederson,2020-09-27,Manchester City,Premier League,3,0,0,19,0,...,0.0,0,0,15,15,100.0,5,0,0,Ederson Santana de Moraes
2,2020-2021,Ederson,2020-10-03,Manchester City,Premier League,4,0,0,36,0,...,0.0,0,0,24,30,80.0,17,0,0,Ederson Santana de Moraes
3,2020-2021,Ederson,2020-10-17,Manchester City,Premier League,5,0,0,50,0,...,0.0,0,0,29,46,63.0,29,0,0,Ederson Santana de Moraes
4,2020-2021,Ederson,2020-10-24,Manchester City,Premier League,6,0,0,26,0,...,0.0,0,0,24,26,92.3,13,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8049,2020-2021,Lys Mousset,2021-03-14,Sheffield United,Premier League,28,0,0,11,6,...,0.0,0,0,5,6,83.3,4,0,1,Lys Mousset
8050,2020-2021,Jack O'Connell,2020-09-14,Sheffield United,Premier League,1,0,0,62,7,...,0.0,0,0,41,54,75.9,40,0,1,Jack O'Connell
8051,2020-2021,Jack O'Connell,2020-09-21,Sheffield United,Premier League,2,0,0,34,3,...,0.0,0,0,19,23,82.6,10,0,0,Jack O'Connell
8052,2020-2021,Iliman Ndiaye,2021-03-14,Sheffield United,Premier League,28,0,0,6,8,...,0.0,0,0,3,3,100.0,4,0,0,Iliman Ndiaye


In [27]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [34]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [36]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [26]:
# any additional corrections
# match_list['Khanya Leshabela'] = 'Thakgalo Leshabela'
# match_list['Okay Yokuşlu'] = 'Okay Yokuslu'

# match_list['Abdoulaye Doucour�'] = 'Abdoulaye Doucouré'
# match_list['James Rodr�guez'] = 'James Rodríguez'
# match_list['Gylfi Sigur�sson'] = 'Gylfi Sigurdsson'
# match_list['Andr� Gomes'] = 'André Filipe Tavares Gomes'
# match_list['S�amus Coleman'] = 'Seamus Coleman'
# match_list['Emiliano Mart�nez'] = 'Emiliano Martínez'
# match_list['Bertrand Traor�'] = 'Bertrand Traoré'
# match_list['Tr�z�guet'] = 'Mahmoud Ahmed Ibrahim Hassan'
# match_list['H�lder Costa'] = 'Hélder Wander Sousa de Azevedo e Costa'
# match_list['Pablo Hern�ndez'] = 'Pablo Hernández Domínguez'
# match_list['Rui Patr�cio'] = 'Rui Pedro dos Santos Patrício'
# match_list['N�lson Semedo'] = 'Nélson Cabral Semedo'
# match_list['R�ben Neves'] = 'Rúben Diogo da Silva Neves'
# match_list['Jo�o Moutinho'] = 'João Filipe Iria Santos Moutinho'
# match_list['Adama Traor�'] = 'Adama Traoré'
# match_list['Romain Sa�ss'] = 'Romain Saïss'
# match_list['Rayan A�t Nouri'] = 'Rayan Ait Nouri'
# match_list['Fernando Mar�al'] = 'Fernando Marçal'
# match_list['F�bio Silva'] = 'Fabio Silva'
# match_list['Willian Jos�'] = 'Willian José Da Silva'
# match_list['Miguel Almir�n'] = 'Miguel Almirón'

In [27]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [28]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [29]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [37]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [38]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [39]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [40]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [41]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [42]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [43]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [45]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles
106043,Harry Kane,26,4,90,Tottenham Hotspur,Burnley,1.627545,0.342968,True,6,...,0.6,0.2,3,0,32,39,82.1,38,2,2
106044,Harry Kane,26,4,90,Tottenham Hotspur,Fulham,1.627545,0.490763,False,2,...,0.9,0,3,0,21,30,70,26,2,2
107056,Harry Kane,27,4,79,Tottenham Hotspur,Crystal Palace,1.627573,0.504114,True,19,...,1,1.1,4,2,22,31,71,27,0,0
107789,Harry Kane,28,4,90,Tottenham Hotspur,Arsenal,1.586900,1.279916,False,2,...,0.2,0,2,0,10,15,66.7,12,0,0
108337,Harry Kane,29,4,90,Tottenham Hotspur,Aston Villa,1.587455,0.885365,False,7,...,0.3,0.0,2,2,19,27,70.4,29,1,3


In [46]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles
108510,Willian Borges Da Silva,29,3,0,Arsenal,West Ham United,1.280364,0.652307,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108511,Yoshinori Muto,29,4,0,Newcastle United,Brighton and Hove Albion,0.589233,0.579673,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108512,Yves Bissouma,29,3,90,Brighton and Hove Albion,Newcastle United,0.579673,0.589233,True,3,...,0.0,0.0,2,1,80,86,93.0,63,1,1
108513,Zeze Steven Sessegnon,29,2,0,Fulham,Leeds,0.520213,0.500510,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108514,Ørjan Nyland,29,1,0,Aston Villa,Tottenham Hotspur,0.885365,1.587455,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df_latest_further.shape

(277, 56)

In [48]:
df_train_new.shape

(108515, 58)

In [49]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v7.csv')

In [50]:
# df_train_new[(df_train_new['team'] == 'Manchester United') & 
#              (df_train_new['gw'] == 32) & 
#              (df_train_new['season'] == '1920')]

In [53]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)

# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [54]:
fixtures.head(50)

,home_team,away_team,gw,match_no
290,Arsenal,Liverpool,30,0
291,Aston Villa,Fulham,30,1
292,Chelsea,West Bromwich Albion,30,2
293,Everton,Crystal Palace,30,3
294,Leeds,Sheffield United,30,4
295,Leicester City,Manchester City,30,5
296,Manchester United,Brighton and Hove Albion,30,6
297,Newcastle United,Tottenham Hotspur,30,7
298,Southampton,Burnley,30,8
299,Wolverhampton Wanderers,West Ham United,30,9


In [55]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [56]:
fixtures.head()

,gw,match_no,home_team,away_team,home_team_code,away_team_code
0,30,0,Arsenal,Liverpool,3,14
1,30,1,Aston Villa,Fulham,7,54
2,30,2,Chelsea,West Bromwich Albion,8,35
3,30,3,Everton,Crystal Palace,11,31
4,30,4,Leeds,Sheffield United,2,49


In [57]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [58]:
all_players[all_players['play_proba_2021'] == '75']

,full_name,play_proba_2021,position_2021,id_2021,cost_2021,team_2021,web_name_2021
19,Emile Smith Rowe,75,3,23,42,3,Smith Rowe
20,Bukayo Saka,75,3,24,53,3,Saka
25,Willian Borges Da Silva,75,3,478,75,3,Willian
43,Jack Grealish,75,3,37,75,7,Grealish
45,Kortney Hause,75,2,39,44,7,Hause
90,Aaron Connolly,75,4,78,52,36,Connolly
189,Tyrick Mitchell,75,2,146,38,31,Mitchell
214,Tom Davies,75,3,163,52,11,Davies
226,Ben Godfrey,75,2,566,50,11,Godfrey
246,Bobby Decordova-Reid,75,4,181,53,54,Decordova-Reid


In [59]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2021', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'full_name', 'position_2021', 
                   'cost_2021', 'play_proba_2021', 'web_name_2021']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2021':'position',
                        'cost_2021':'price',
                        'play_proba_2021':'play_proba',
                        'web_name_2021':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [60]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2021', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'full_name', 'position_2021', 
                   'cost_2021', 'play_proba_2021', 'web_name_2021']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2021':'position',
                        'cost_2021':'price',
                        'play_proba_2021':'play_proba',
                        'web_name_2021':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [61]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,player,position,price,play_proba,short_name,was_home
0,30,0,Liverpool,Arsenal,James Milner,3,53,100,Milner,False
1,30,0,Liverpool,Arsenal,Konstantinos Tsimikas,2,49,100,Tsimikas,False
2,30,0,Liverpool,Arsenal,Georginio Wijnaldum,3,53,None,Wijnaldum,False
3,30,0,Liverpool,Arsenal,Jordan Henderson,3,54,0,Henderson,False
4,30,0,Liverpool,Arsenal,Adrián San Miguel del Castillo,1,42,None,Adrián,False


In [62]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,player,position,price,play_proba,short_name,was_home
0,30,0,Arsenal,Liverpool,Mesut Özil,3,67,0,Özil,True
1,30,0,Arsenal,Liverpool,Sokratis Papastathopoulos,2,48,0,Sokratis,True
2,30,0,Arsenal,Liverpool,David Luiz Moreira Marinho,2,54,100,David Luiz,True
3,30,0,Arsenal,Liverpool,Pierre-Emerick Aubameyang,3,115,100,Aubameyang,True
4,30,0,Arsenal,Liverpool,Cédric Soares,2,46,100,Cédric,True


In [63]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks in current season
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
#                            (df_train_new['season'] == '1920')) | 
#                            ((df_train_new['gw'] >= 1) & 
#                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron Connolly': 20.0,
 'Aaron Cresswell': 90.0,
 'Aaron Ramsdale': 90.0,
 'Aaron Wan-Bissaka': 90.0,
 'Abdoulaye Doucouré': 60.75,
 'Adam Lallana': 67.4,
 'Adama Traoré': 90.0,
 'Ademipo Odubeko': 0.0,
 'Ademola Lookman': 88.66666666666667,
 'Adrián Bernabé': 0.0,
 'Adrián San Miguel del Castillo': 18.0,
 'Ahmed El Mohamady': 75.0,
 'Ainsley Maitland-Niles': 90.0,
 'Aleksandar Mitrović': 25.666666666666668,
 'Alex Iwobi': 41.8,
 'Alex McCarthy': 45.0,
 'Alex Nicolao Telles': 0.0,
 'Alex Oxlade-Chamberlain': 7.2,
 'Alexandre Jankewitz': 0.0,
 'Alexandre Lacazette': 60.6,
 'Alexis Mac Allister': 38.0,
 'Alfie Devine': 0.0,
 'Alireza Jahanbakhsh': 2.0,
 'Alisson Ramses Becker': 72.0,
 'Allan Marques Loureiro': 60.4,
 'Allan Saint-Maximin': 69.0,
 'Allan Tchaptchet': 0.0,
 'Alphonse Areola': 90.0,
 'Amad Diallo': 0.0,
 'Andi Zeqiri': 10.4,
 'Andreas Christensen': 72.0,
 'Andreas Söndergaard': 0.0,
 'Andrew Robertson': 90.0,
 'Andros Townsend': 68.0,
 'André Filipe Tavares Gomes': 79.8,


In [64]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2021'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2021'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last five fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [65]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,30,0,Arsenal,Liverpool,Mesut Özil,3,6.7,0.00,Özil,True,1.280364,2.355067,2021,0.000000
1,30,0,Arsenal,Liverpool,Sokratis Papastathopoulos,2,4.8,0.00,Sokratis,True,1.280364,2.355067,2021,0.000000
2,30,0,Arsenal,Liverpool,David Luiz Moreira Marinho,2,5.4,1.00,David Luiz,True,1.280364,2.355067,2021,73.600000
3,30,0,Arsenal,Liverpool,Pierre-Emerick Aubameyang,3,11.5,1.00,Aubameyang,True,1.280364,2.355067,2021,53.400000
4,30,0,Arsenal,Liverpool,Cédric Soares,2,4.6,1.00,Cédric,True,1.280364,2.355067,2021,36.000000
5,30,0,Arsenal,Liverpool,Alexandre Lacazette,4,8.2,1.00,Lacazette,True,1.280364,2.355067,2021,60.600000
6,30,0,Arsenal,Liverpool,Shkodran Mustafi,2,5.0,0.00,Mustafi,True,1.280364,2.355067,2021,0.000000
7,30,0,Arsenal,Liverpool,Bernd Leno,1,5.0,1.00,Leno,True,1.280364,2.355067,2021,90.000000
8,30,0,Arsenal,Liverpool,Granit Xhaka,3,5.2,1.00,Xhaka,True,1.280364,2.355067,2021,86.600000
9,30,0,Arsenal,Liverpool,Pablo Marí,2,4.4,1.00,Marí,True,1.280364,2.355067,2021,72.000000


In [66]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')